# Content Reconstruction by Layer

In [ ]:
import numpy as np
np.random.seed(222)
import tensorflow as tf
tf.random.set_seed(222)
from matplotlib import pyplot as plt
%matplotlib inline
import skimage.io as io
import skimage.transform as T

In [ ]:
from src.reconstruct_content import reconstruct_content
from src.optimizers import GradientDescent, Adam, L_BFGS
from src.callback import Callback

In [ ]:
ITERATIONS = 30

In [ ]:
plt.figure(figsize=(15, 7))
plt.subplot(1, 2, 1)
content = io.imread('img/content/tubingen.jpg')
ax = plt.imshow(content)
plt.title('Content Image $p$')
ax.axes.xaxis.set_major_locator(plt.NullLocator())
ax.axes.yaxis.set_major_locator(plt.NullLocator())
plt.subplot(1, 2, 2)
noise = np.random.uniform(0, 1, size=content.shape)
ax = plt.imshow(noise)
plt.title('White Noise Image $x$')
ax.axes.xaxis.set_major_locator(plt.NullLocator())
ax.axes.yaxis.set_major_locator(plt.NullLocator())

## `block1_conv1`

In [ ]:
output = reconstruct_content(
    content=content,
    layer_name='block1_conv1',
    optimize=L_BFGS(),
    iterations=ITERATIONS,
    callback=Callback('build/content/block1_conv1')
)
io.imsave('img/content-reconstruction/block1_conv1.jpg', output)

## `block2_conv1`

In [ ]:
output = reconstruct_content(
    content=content,
    layer_name='block2_conv1',
    optimize=L_BFGS(),
    iterations=ITERATIONS,
    callback=Callback('build/content/block2_conv1')
)
io.imsave('img/content-reconstruction/block2_conv1.jpg', output)

## `block3_conv1`

In [ ]:
output = reconstruct_content(
    content=content,
    layer_name='block3_conv1',
    optimize=L_BFGS(),
    iterations=ITERATIONS,
    callback=Callback('build/content/block3_conv1')
)
io.imsave('img/content-reconstruction/block3_conv1.jpg', output)

## `block4_conv1`

In [ ]:
output = reconstruct_content(
    content=content,
    layer_name='block4_conv1',
    optimize=L_BFGS(),
    iterations=ITERATIONS,
    callback=Callback('build/content/block4_conv1')
)
io.imsave('img/content-reconstruction/block4_conv1.jpg', output)

## `block5_conv1`

In [ ]:
output = reconstruct_content(
    content=content,
    layer_name='block5_conv1',
    optimize=L_BFGS(),
    iterations=ITERATIONS,
    callback=Callback('build/content/block5_conv1')
)
io.imsave('img/content-reconstruction/block5_conv1.jpg', output)

## `block4_conv2`

**Gatys et al. use `block4_conv2` as the final content representation.**

In [ ]:
output = reconstruct_content(
    content=content,
    layer_name='block4_conv2',
    optimize=L_BFGS(),
    iterations=ITERATIONS,
    callback=Callback('build/content/block4_conv2')
)
io.imsave('img/content-reconstruction/block4_conv2.jpg', output)